## Exploring information from student grades in 902, 10th, and 11th grade.

In [1]:
from utils import parsing_html
import pandas as pd
import os

In [2]:
# prettifying all HTML files in the directory.
raw_dir = "./grades_html"
for file in os.listdir(raw_dir):
    parsing_html.prettify_html(os.path.join(raw_dir, file))

In [5]:
# Define cleaned parque dataframes path.
cleaned_data_dir = "./cleaned_data"
if not os.path.exists(cleaned_data_dir):
    os.makedirs(cleaned_data_dir, exist_ok=True)

In [6]:
new_parquet_file = "10_4.parquet"
parsing_html.full_grades_table("./grades_html/10_4_2grades.html").to_parquet(path=os.path.join(cleaned_data_dir, new_parquet_file))